
# **Final Project Documentation: GPS Motion Analysis**

This Jupyter Notebook contains the documentation for the final project. It includes an analysis of GPS motion data collected using the Phyphox app, with corresponding figures generated through Python functions implemented in `final.py`.

---

## **How to Run the Code**

1. **Set Up the Environment:**
   - Clone the repository and navigate to the project folder.
   - Install the required dependencies using `pip install -r code/requirements.txt`.

2. **Run This Notebook:**
   - Launch Jupyter Notebook with `jupyter notebook documentation/final.ipynb`.

3. **Expected Output:**
   - Figures of GPS motion analysis, FFT analysis, and inverse FFT.

---

## **Bibliography**

- Phyphox App Documentation: [https://phyphox.org](https://phyphox.org)
- SciPy Documentation: [https://scipy.org](https://scipy.org)
- NumPy Documentation: [https://numpy.org](https://numpy.org)
- Matplotlib Documentation: [https://matplotlib.org](https://matplotlib.org)


In [8]:

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from final import (
    parse_temperature_from_markdown,
    list_markdown_files,
    non_linear_fit,
    fft_with_check,
    inverse_fft,
    calculate_frequency_axis
)


ModuleNotFoundError: No module named 'final'

In [ ]:

# Load Data and Plot GPS Motion
data_folder = "data"
markdown_files = list_markdown_files(data_folder, keyword="sinewalk")

plt.figure(figsize=(12, 8))
for file in markdown_files:
    # Read the temperature from the markdown file
    temperature_f = parse_temperature_from_markdown(file)
    temperature_k = (temperature_f - 32) * 5 / 9 + 273.15

    # Simulate GPS data loading from a CSV file
    csv_file = file.replace(".md", ".csv")
    if os.path.exists(csv_file):
        data = np.loadtxt(csv_file, delimiter=",", skiprows=1)
        x_data, y_data = data[:, 0], data[:, 1]

        # Perform non-linear fitting
        initial_guess = (1, 1, 0, 0)
        popt, _ = non_linear_fit(x_data, y_data, initial_guess)

        # Plot GPS motion and the fit
        plt.scatter(x_data, y_data, label=f"Run {file} (T={temperature_k:.2f} K)", alpha=0.6)
        plt.plot(x_data, popt[0] * np.sin(popt[1] * x_data + popt[2]) + popt[3], color="red")

plt.title("GPS Motion with Non-Linear Fit")
plt.xlabel("Position (m)")
plt.ylabel("Displacement (m)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Perform FFT on GPS Data
plt.figure(figsize=(12, 8))
for file in markdown_files:
    csv_file = file.replace(".md", ".csv")
    if os.path.exists(csv_file):
        data = np.loadtxt(csv_file, delimiter=",", skiprows=1)
        x_data, y_data = data[:, 0], data[:, 1]

        # Perform FFT
        fft_y, freq = fft_with_check(x_data, y_data)

        # Plot FFT
        plt.plot(calculate_frequency_axis(len(y_data), x_data[1] - x_data[0]), np.abs(fft_y[:len(y_data)//2]), label=f"Run {file}")

plt.title("FFT Analysis of GPS Walks")
plt.xlabel("Frequency (1/100m)")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Perform Inverse FFT on the Average of All Walks
mean_y = np.mean([np.loadtxt(file.replace(".md", ".csv"), delimiter=",", skiprows=1)[:, 1] for file in markdown_files], axis=0)

fft_mean, _ = fft_with_check(np.arange(len(mean_y)), mean_y)
inv_fft = inverse_fft(fft_mean)

plt.figure(figsize=(12, 8))
plt.plot(mean_y, label="Mean Sine Walk Data", color="blue")
plt.plot(inv_fft, label="Inverse FFT of Mean", color="orange")
plt.title("Inverse FFT of Filtered Mean Value")
plt.xlabel("Position (m)")
plt.ylabel("Displacement (m)")
plt.legend()
plt.grid(True)
plt.show()
